In [1]:
import telebot
import xlrd
import psycopg2
import warnings
from tqdm.notebook import tqdm
from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
nltk.download('stopwords')

import os
import re
import time
import random
import datetime

import pymorphy2
import tensorflow as tf

from nltk.corpus import words
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from autocorrect import Speller

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

from matplotlib import pyplot as plt

from datetime import datetime, timedelta

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/darya.mishina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## загружаем данные для обучения модели

### вариант загрузки из файла

In [4]:
df = pd.read_csv('file.csv')

### вариант загрузки из БД

In [ ]:
#убрала параметры подключения из-за конфиденциальности
config = {"conn_string": {"host": "", "port": "", "dbname": "",
                         "user": "", "password": ""}}

In [ ]:
def read_sql(sql, error=0, conf = config):
    conn = psycopg2.connect(**conf.get('conn_string'))
    try:
        df = pd.read_sql(sql,conn)
        conn.close()
        return df
    except Exception as t:
        print(t)
        if error<100:
            error+=1
            conn.close()
            time.sleep(1.5)
            return read_sql(sql = sql,error = error,conf = conf)
        else:
            conn.close()

In [ ]:
#убрала скрипт из-за конфиденциальности
query = f"""
SELECT 
    
FROM 

WHERE 

    """

In [ ]:
df = read_sql(query)

## классы

In [5]:
def classes_define(data):
    target_data = data[data['Категория обращения 2'].isna()!=True][['Подробное описание','Категория обращения 2']]
    enc = LabelEncoder()
    target_data['teg'] = enc.fit_transform(target_data['Категория обращения 2'])
    a = target_data.groupby('teg').teg.count().sort_values()
    classes = list(a[a>20].index)
    target_data['teg'] = target_data['teg'].apply(lambda x: x if x in classes else 99)
    target_data_1 = target_data
    return target_data, target_data_1, classes

In [6]:
target_data, target_data_1, classes = classes_define(df)

## подготовка для TF-IDF

In [7]:
def preprocess_text(text):
    text = str(text)
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and len(token)>=3 \
              and token.strip() not in punctuation \
              and token.isdigit()==False]
    text = " ".join(tokens)
    return text

In [8]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['лента','ассорт','разм','арт','что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на'])

In [9]:
def tfidf_preproc(target_data_1):
    target_data_1['Подробное описание_11'] = target_data_1['Подробное описание'].apply(lambda x: re.sub(r'[^a-zA-Zа-яА-Я 0-9]+','',x))
    target_data_1['processed_lem']=target_data_1['Подробное описание_11'].apply(preprocess_text)
    x_train, x_test, y_train, y_test = train_test_split(target_data_1['processed_lem'],target_data_1['teg'], test_size = 0.5)
    return x_train, x_test, y_train, y_test

In [10]:
x_train, x_test, y_train, y_test = tfidf_preproc(target_data_1)

## модель

In [11]:
def tfidf_model(x_train, y_train):
    
    tf_idf = TfidfVectorizer(stop_words=russian_stopwords, max_features=50000, min_df=2)
    
    logit = LogisticRegression(C=1e2, n_jobs=4, solver='lbfgs', 
                           random_state=17, verbose=0, 
                           multi_class='multinomial',
                           fit_intercept=True, max_iter=400)
   
    tfidf_logit_pipeline = Pipeline([('tf_idf', tf_idf), 
                                 ('logit', logit)])
    tfidf_logit_pipeline.fit(x_train, y_train)
    return tfidf_logit_pipeline

In [12]:
tfidf_logit_pipeline = tfidf_model(x_train, y_train)

## файл, который в бот отправляют пользователи

In [21]:
valid = pd.read_excel('Декабрь.xlsx', engine='openpyxl')

## обрабатываем файл из бота

In [24]:
def test_preproc(valid):
    valid['Подробное описание'] = valid['Подробное описание'].apply(str)
    valid['Подробное описание_11'] = valid['Подробное описание'].apply(lambda x: re.sub(r'[^a-zA-Zа-яА-Я 0-9]+','',x))
    valid['processed_lem']=valid['Подробное описание_11'].apply(preprocess_text)
    return valid

In [25]:
valid = test_preproc(valid)

## Предсказываем

In [26]:
def predict_classes(valid, tfidf_logit_pipeline):
    x_valid = valid['processed_lem']
    valid_pred_final = tfidf_logit_pipeline.predict(x_valid)
    valid['teg'] = valid_pred_final
    return valid

In [27]:
valid = predict_classes(valid, tfidf_logit_pipeline)

## подтягиваем к тегу категории

In [28]:
def cat_add(target_data, data, classes):
    name1_teg = target_data[['Категория обращения 2', 'teg']]
    name1_teg = name1_teg.drop_duplicates()
    
    name2_teg =data[['Категория обращения 1', 'Категория обращения 2']]
    name2_teg = name2_teg.drop_duplicates()
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Проблемы в работе оборудования'] = "3. Оборудование"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Проблемы с товарами'] = "2. BO"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Настройка рабочего места'] = "3. Оборудование"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Сетевые папки'] = "4. Программное обеспечение"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Редактирование заказов'] = "2. BO"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Уточнение статуса или решения'] = "6. Запрос решения по обращению"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Проблемы с бонусными баллами и картами лояльности'] = "7. www.vprok.ru"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Проблемы в работе приложения'] = "8. Мобильное приложение"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Учётные записи'] = "3. Оборудование"
    name2_teg['Категория обращения 1'][name2_teg['Категория обращения 2'] == 'Проблемы с адресами доставки'] = "7. www.vprok.ru"
    name2_teg = name2_teg.drop_duplicates()
    name_teg = pd.merge(name1_teg, 
                      name2_teg, 
                      on ='Категория обращения 2', 
                      how ='inner')

    name_teg['teg'] = name_teg['teg'].apply(lambda x: x if x in classes else 99)
    #меняем название тэгов 99 на прочее
    name_teg['Категория обращения 2'][name_teg.teg == 99] = "Прочее"
    name_teg['Категория обращения 1'][name_teg.teg == 99] = "Прочее"
    name_teg = name_teg.drop_duplicates()
    return name_teg

In [29]:
name_teg = cat_add(target_data, df, classes)

In [30]:
final = pd.merge(valid, 
                    name_teg, 
                   on ='teg', 
                   how ='inner')
final.drop(columns=['Подробное описание_11', 'processed_lem'], inplace=True)

## или выгружаем в эксель

In [31]:
path_to_save = ''
filename = 'tickets_support_classification_Декабрь_3.xlsx' 
final.to_excel(os.path.join(path_to_save, filename), index=False)

## или пишем в БД

In [ ]:
#функция ктр записывает данные
def metrics(df, table, varible = 'date', drop = False):
    if df.empty == False:
        df[varible] = pd.to_datetime(df[varible])
        d1 = df[varible].min().strftime("%Y-%m-%d %H:%M:%S")
        d2 = (df[varible].max()).strftime("%Y-%m-%d %H:%M:%S")
        condition = datetime.now()
        df['version'] = condition
        conn = psycopg2.connect(**config.get("conn_mybase"))
        df_columns = list(df)
        columns = ",".join(df_columns)
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns]))
        cur = conn.cursor()
        insert_stmt = "INSERT INTO {} ({}) {}".format(table, columns, values)
        if drop!=False:
            drop_table = "DELETE from {}".format(table)
            cur.execute(drop_table)
        execute_batch(cur, insert_stmt, df.values)
        conn.commit()
        cur.close()
        conn.close()  





#вот здесь указываем схему и таблицу будет 
metrics(final, 'scheme.table', varible = 'date')

## или отправляем все в бот

In [52]:
TOKEN  = '5063.......' #убрала часть токена

bot = telebot.TeleBot(TOKEN)


### Прием документов
@bot.message_handler(content_types=['document'])

def handle_docs(message):
    if message.chat.username in ['Daria_Mishina28']: #здесь указать публичные имена пользователей в тг
        print('try1')
        try:
            file_info = bot.get_file(message.document.file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            print('downloaded_file compplete')
            #отдельная папка для присланных файлов
            src = 'data/' +message.chat.username +'_'+ str(datetime.now()) +'_'+ message.document.file_name 
            with open(src, 'wb') as new_file:
                new_file.write(downloaded_file)
            try:
                print(src)
                data = pd.read_excel(src)
                print('data загружена')
                #файл для обучения
                df = pd.read_csv('Статистика по 1 линии (Q2) - размеченные данные.csv')
                print('df загружен')
                #классы
                target_data, target_data_1, classes = classes_define(df)
                print('классы готовы')
                #подготовка для TF-IDF
                x_train, x_test, y_train, y_test = tfidf_preproc(target_data_1)
                print('x_train, x_test готов')
                #модель
                tfidf_logit_pipeline = tfidf_model(x_train, y_train)
                print('модель готова')
                #обрабатываем файл из тг
                valid = test_preproc(data)
                print('файл из тг обработан')
                #предсказываем
                valid = predict_classes(valid, tfidf_logit_pipeline)
                print('предсказано')
                #обрабатываем предсказания для выгрузки
                name_teg = cat_add(target_data, df, classes)
                print('категории сформированы')
                #подтягиваем категории к предсказаниям
                final = pd.merge(valid, 
                      name_teg, 
                      on ='teg', 
                      how ='inner')
                final.drop(columns=['Подробное описание_11', 'processed_lem'], inplace=True)
                print('категории соединены с предсказанием')
                src2 = 'data/' + 'new_' + message.chat.username +'_'+ str(datetime.now()) + '_' + message.document.file_name
                final.to_excel(src2, index=False)
                print('предсказания готовы')
                f = open(src2, 'rb')
                bot.send_document(chat_id=message.chat.id, data=f, disable_notification=True) 
                print('файл отправлен в бот')
                f.close()
                #os.remove(src) #загруженные и обработанные файлы не удаляем
                #os.remove(src2)
            except:
                bot.reply_to(message, "что-то пошло не так")
        except Exception as e:
            bot.reply_to(message, e)
    else:
        pass

In [53]:
bot.polling()

try1
downloaded_file compplete
data/Daria_Mishina28_2021-12-24 16:38:29.063250_Прошлая неделя.xlsx
data загружена
df загружен
классы готовы
x_train, x_test готов
модель готова
файл из тг обработан
предсказано
категории сформированы
категории соединены с предсказанием
предсказания готовы
файл отправлен в бот
